In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [6]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
images_df= sc.binaryFiles("/ADACA1/futbol,/ADACA1/Rugby")

In [8]:
images_df

org.apache.spark.api.java.JavaPairRDD@d4b494d

In [9]:
from PIL import Image
import io

image_data = images_df.collect()
image_pixels = []
image_labels = []

for image_path, image_byte in image_data:
    image = Image.open(io.BytesIO(image_byte))
    pixels = list(image.getdata())
    
    image_pixels.append(pixels)
    image_labels.append(image_path.split('/')[-1])  

df = pd.DataFrame({'image': image_pixels, 'label': image_labels})

In [10]:
df.head()

,image,label
0,"[(193, 185, 148), (193, 185, 148), (194, 186, ...",futbol_001.jpg
1,"[(25, 33, 20), (22, 30, 17), (22, 30, 19), (26...",futbol_002.jpeg
2,"[(58, 64, 54), (67, 73, 63), (52, 55, 46), (55...",futbol_003.jpeg
3,"[(0, 0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0), (0,...",futbol_004.png
4,"[(208, 196, 172), (208, 196, 172), (208, 196, ...",futbol_005.jpg


In [11]:
df.shape

(42, 2)

In [12]:
type(df)

pandas.core.frame.DataFrame

In [13]:
#normalize the data
df['image'] = df['image'].apply(lambda x: [pixel/255 if isinstance(pixel, int) else tuple(p/255 for p in pixel) for pixel in x])

In [14]:
df.head()

,image,label
0,"[(0.7568627450980392, 0.7254901960784313, 0.58...",futbol_001.jpg
1,"[(0.09803921568627451, 0.12941176470588237, 0....",futbol_002.jpeg
2,"[(0.22745098039215686, 0.25098039215686274, 0....",futbol_003.jpeg
3,"[(0.0, 0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0), (...",futbol_004.png
4,"[(0.8156862745098039, 0.7686274509803922, 0.67...",futbol_005.jpg


In [15]:

target_size = (100, 100)

# Function to resize or crop images
def resize_image(image_data):
    image = Image.fromarray(np.array(image_data))  # Convert list to array
    image = image.resize(target_size)
    return np.array(image)

# Apply the resize_image function to the 'image' column in the DataFrame
df['image'] = df['image'].apply(resize_image)

In [16]:
from sklearn.model_selection import train_test_split

# Define features (X) and labels (y)
X = np.array(df['image'].tolist())
y = df['label']

# Split the data into training and testing sets (e.g., 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Training set shape: (33, 100, 100) (33,)
Testing set shape: (9, 100, 100) (9,)


In [2]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip show tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras import layers, models